Import all the needed libraries

In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error    
from sklearn.preprocessing import PolynomialFeatures    # function to generate polynomial and interaction features
from sklearn.linear_model import LinearRegression    # classes providing Linear Regression with ordinary squared error loss and Huber loss, respectively
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

Read the data

In [19]:
df = pd.read_csv('vehicles.csv')

Check if the correct data as imported

In [20]:
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.drop((df['price'] == 0).index)
df.dropna(subset = ['price','year','odometer','type','fuel','transmission','cylinders','drive'], inplace=True)
df = df[['price','year','type','manufacturer','odometer', 'fuel', 'transmission']]

In [21]:
df = df[df['manufacturer'] == 'ford']
df = df[df['type'] == 'truck']
df = df.replace('gas',0)
df = df.replace('diesel',1)
df = df.replace('hybrid',2)
df = df.replace('other',3)
df = df.replace('electric',4)
df = df.replace('manual',0)
df = df.replace('automatic',1)
df = df.replace('other',2)

In [22]:
price = df['price']
year = df['year']
distance = df['odometer']
fuel = df['fuel']
transmission = df['transmission']

In [23]:
price = price.to_numpy()
year = year.to_numpy()
distance = distance.to_numpy()
fuel = fuel.to_numpy()
transmission = transmission.to_numpy()

In [24]:
df = df[['price','year','odometer', 'fuel', 'transmission']]

In [25]:
display(df)

,price,year,odometer,fuel,transmission
31,15000,2013.0,128000.0,0,1
65,22500,2001.0,144700.0,1,0
175,12500,2008.0,141345.0,0,1
177,22950,2014.0,166380.0,0,1
337,8950,2011.0,164000.0,0,1
...,...,...,...,...,...
426692,56988,2019.0,18671.0,0,1
426716,65950,2020.0,21300.0,1,1
426766,55995,2019.0,23670.0,0,1
426776,52995,2019.0,31558.0,0,1


In [26]:
features = []
labels = []
m = 0
for row in df.iterrows() :
    datapoint_one = distance[m].astype('int')
    datapoint_two = year[m].astype('int')
    datapoint_three = fuel[m]
    datapoint_four = transmission[m]
    label_one = price[m]
    features.append([datapoint_one, datapoint_two,datapoint_three,datapoint_four])
    labels.append(label_one)
    m = m + 1
print(m)

9564


In [27]:
X = np.array(features)
y = np.array(labels)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state = 1)
X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.5, random_state = 1)

In [29]:
regressor = DecisionTreeRegressor(criterion = 'squared_error',random_state=1)
regressor.fit(X_train, y_train)
y_pred_val= regressor.predict(X_val)
y_pred_train = regressor.predict(X_train)
accuracy_val = accuracy_score(y_val,y_pred_val.astype('int'))
accuracy_train = accuracy_score(y_train,y_pred_train.astype('int'))

In [34]:
y_reg= regressor.predict(X_test)

In [35]:
acc_reg = accuracy_score(y_test,y_reg.astype('int'))

In [36]:
lin = LinearRegression()
lin.fit(X_train,y_train)
y_lin = lin.predict(X_val)
acc = accuracy_score(y_val,y_lin.astype('int'))

In [37]:
poly = PolynomialFeatures(degree= 10)    # initialize a polynomial feature transformer 
X_poly = poly.fit_transform(X_train)    # fit and transform the raw features
    
lin_regr = LinearRegression(fit_intercept=True) # NOTE: "fit_intercept=False" as we already have a constant iterm in the new feature X_poly
lin_regr.fit(X_poly,y_train)# fit linear regression to these new features and labels (labels remain same)
X_val = poly.fit_transform(X_val) 
    
y_lin = lin_regr.predict(X_val)    # predict using the learnt linear model
tr_error = mean_squared_error(y_val,y_lin)    # calculate the training error
acc = accuracy_score(y_val,y_lin.astype('int'))